## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-02-13-26 +0000,nyt,Trump-Style Candidate Heads to Runoff in Chile...,https://www.nytimes.com/2025/11/16/world/ameri...
1,2025-11-17-02-11-18 +0000,nyt,U.S. Airport Restrictions to End Monday,https://www.nytimes.com/2025/11/16/us/politics...
2,2025-11-17-02-05-50 +0000,nyt,"Latest U.S. Strike on Boat in Pacific Kills 3,...",https://www.nytimes.com/2025/11/16/us/politics...
3,2025-11-17-02-00-00 +0000,wsj,"Layoff Tactics Keep Changing, and the Blunders...",https://www.wsj.com/business/layoff-job-cut-st...
4,2025-11-17-02-00-00 +0000,wsj,Wall Street Blows Past Bubble Worries to Super...,https://www.wsj.com/finance/investing/wall-str...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,20
431,epstein,9
227,new,8
404,will,7
107,up,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
79,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...,57
113,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,55
147,2025-11-16-08-12-45 +0000,bbc,Feud erupts between Trump and ally Marjorie Ta...,https://www.bbc.com/news/articles/c9wv4dx05q5o...,54
59,2025-11-16-21-32-28 +0000,bbc,Marjorie Taylor Greene doubles down on Epstein...,https://www.bbc.com/news/articles/cdeg8z0j2j2o...,53
120,2025-11-16-14-50-10 +0000,nypost,Marjorie Taylor Greene insists she still suppo...,https://nypost.com/2025/11/16/us-news/marjorie...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
79,57,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...
23,36,2025-11-17-00-19-40 +0000,nypost,"Liberal pols, DSA warn Gov. Kathy Hochul about...",https://nypost.com/2025/11/16/us-news/liberals...
39,30,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
28,28,2025-11-17-00-00-21 +0000,bbc,Formula 1 tech used to rev-up train wi-fi spee...,https://www.bbc.com/news/articles/cqjwkdez71ro...
16,26,2025-11-17-00-51-22 +0000,nypost,22-year-old ID’d as gunman killed in shootout ...,https://nypost.com/2025/11/16/us-news/gunman-k...
125,25,2025-11-16-13-33-18 +0000,bbc,The US will help South Korea build nuclear 'at...,https://www.bbc.com/news/articles/c620qppzlgwo...
112,25,2025-11-16-17-00-00 +0000,wsj,Over 60% of Americans are comfortable with how...,https://www.wsj.com/politics/policy/economic-o...
48,23,2025-11-16-22-45-28 +0000,nypost,Canal Street knock-off vendors back with venge...,https://nypost.com/2025/11/16/us-news/nyc-knoc...
83,23,2025-11-16-19-47-46 +0000,cbc,U.S. stations its most advanced aircraft carri...,https://www.cbc.ca/news/world/us-venezuela-air...
120,22,2025-11-16-14-50-10 +0000,nypost,Marjorie Taylor Greene insists she still suppo...,https://nypost.com/2025/11/16/us-news/marjorie...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
